Goal of this notebook: <br> To classify whether the image contains damaged or un-dmages cars.<br>
Approach: Deep learning <br>
Framework used: Tensorflow<br>

In this notebook, Densenet121 pre trained model is used to classify where the image contain the damaged car or not.

In [2]:
!pip install tensorflow

     |████████████████████████████████| 497.6 MB 2.6 kB/s eta 0:00:011     |██████████████▉                 | 230.4 MB 6.3 MB/s eta 0:00:43     |████████████████▎               | 253.9 MB 2.1 MB/s eta 0:01:58     |████████████████▌               | 255.7 MB 1.4 MB/s eta 0:02:57     |██████████████████████████▉     | 417.6 MB 10.7 MB/s eta 0:00:08
     |████████████████████████████████| 462 kB 13 kB/s  eta 0:00:01
     |████████████████████████████████| 2.1 MB 6.7 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 194 kB/s  eta 0:00:01
     |████████████████████████████████| 4.3 MB 66 kB/s  eta 0:00:01
     |████████████████████████████████| 1.0 MB 9.1 MB/s eta 0:00:01
     |████████████████████████████████| 126 kB 7.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 76 kB/s  eta 0:00:01
     |████████████████████████████████| 42 kB 189 kB/s  eta 0:00:01
     |████████████████████████████████| 5.8 MB 9.3 kB/s eta 0:00:01
     |████████████████████████████████| 

In [4]:
#import necessary librabry
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
#defining the base, train and validation directory path
base_dir = './data1a'
train_dir = os.path.join(base_dir, 'training')
validation_dir = os.path.join(base_dir, 'validation')

In [6]:
#defining the damage and whole , train nand validation directory
train_damage_dir = os.path.join(train_dir, '00-damage')
train_whole_dir = os.path.join(train_dir, '01-whole')
validation_damage_dir = os.path.join(validation_dir, '00-damage')
validation_whole_dir = os.path.join(validation_dir, '01-whole')

Data Augmentation is used to:
1. Rescale the data 
2. to bring all the image to same dimension  i.e. 150x150


In [ ]:
#data augmentation
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
train_generator = train_datagen.flow_from_directory(
        train_dir,  
        target_size=(150, 150), 
        batch_size=20,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 1840 images belonging to 2 classes.
Found 460 images belonging to 2 classes.


For training 1840 images are used.
For Validation 460 image are used.

Training and evaluating the model with binary crossentropy loss, accuracy, precision , recall

In [ ]:
#defining model
from tensorflow.keras import Model 
from tensorflow.keras.applications import DenseNet121   

base_model = DenseNet121(input_shape = (150, 150, 3),  include_top = False, weights = 'imagenet') 

x=   tf.keras.layers.Flatten()(base_model.output)
x=   tf.keras.layers.Dense(512, activation='relu')(x) 
x=   tf.keras.layers.Dense(1, activation='sigmoid')(x) 

model= Model( base_model.input, x)

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=1e-4),
              metrics=['Accuracy','Precision','Recall'])

#training the model
history = model.fit(
      train_generator,
      epochs=15,
      validation_data=validation_generator,
      verbose=2)

2022-04-11 13:48:35.170458: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-04-11 13:48:35.170570: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dell): /proc/driver/nvidia/version does not exist
2022-04-11 13:48:35.171551: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


29097984/29084464 [==============================] - 2s 0us/step


/home/akash/anaconda3/lib/python3.9/site-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/15
92/92 - 640s - loss: 0.5164 - Accuracy: 0.8549 - precision: 0.8584 - recall: 0.8500 - val_loss: 1.0329 - val_Accuracy: 0.7587 - val_precision: 0.6745 - val_recall: 1.0000 - 640s/epoch - 7s/step
Epoch 2/15
92/92 - 361s - loss: 0.1440 - Accuracy: 0.9467 - precision: 0.9497 - recall: 0.9435 - val_loss: 0.3080 - val_Accuracy: 0.9348 - val_precision: 0.9630 - val_recall: 0.9043 - 361s/epoch - 4s/step
Epoch 3/15


In [ ]:
#weights saving
model.save("classifier.h5")

plotting the history of train and validation loss, accurcy, precision, recall

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['Accuracy']
val_acc = history.history['val_Accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['precision']
val_acc = history.history['val_precision']
loss = history.history['recall']
val_loss = history.history['val_recall']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

Testing

In [ ]:
import cv2
im = cv2.imread("./data1a/validation/00-damage/0001.JPEG")
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
im = cv2.resize(im, (150, 150)) 
imS=im.reshape([1,150,150,3])
pred=model.predict(np.array(imS))
if pred[0][0]>0.5:
    print("The car is damaged")
else:
    print("The car is not damaged")
plt.axis("off")
plt.imshow(im)

In [ ]:
im = cv2.imread("./data1a/validation/01-whole/0006.jpg")
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
im = cv2.resize(im, (150, 150)) 
imS=im.reshape([1,150,150,3])
pred=model.predict(np.array(imS))
print(pred)
if pred[0][0]>0.5:
    print("The car is damaged")
else:
    print("The car is not damaged")
plt.axis("off")
plt.imshow(im)


Conclusion: Successfully build a classifier to classify the images containing into damage and undamage cars. 